In [ ]:
# =============================================================================
# MEMORY OPTIMIZATION AND TROUBLESHOOTING
# =============================================================================

import torch
import gc

# Clear GPU memory
print("🧹 Clearing GPU memory...")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    
    # Check current memory usage
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    free = total_memory - allocated
    
    print(f"📊 GPU Memory Status:")
    print(f"   Total: {total_memory:.2f} GB")
    print(f"   Allocated: {allocated:.2f} GB")
    print(f"   Reserved: {reserved:.2f} GB") 
    print(f"   Free: {free:.2f} GB")
    
    if allocated > 25:  # More than 25GB in use
        print("⚠️  High memory usage detected!")
        print("🔧 Suggestions:")
        print("   1. Restart kernel to clear all memory")
        print("   2. Use CPU offloading")
        print("   3. Use 8-bit or 4-bit quantization")
        print("   4. Use smaller model variants")
else:
    print("❌ CUDA not available")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "openai/gpt-oss-20b"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# prompt_low = "Question: What is 25 * 32?\nAnswer:"
# prompt_medium = "Question: What is 25 * 32?\nLet's think step by step.\nAnswer:"
# prompt_high = """{"id":"abc123","conversations":[
# {"from":"user","value":"What is 25 * 32?"},
# {"from":"assistant","value":"Let's think carefully.","scratchpad":"..."}
# ]}"""

# inputs = tokenizer(prompt_medium, return_tensors="pt").to("cuda")
# outputs = model.generate(**inputs, max_new_tokens=128)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
# =============================================================================
# DEPENDENCIES AND IMPORTS (FIXED)
# =============================================================================

import sys
import os

# Add project root to path (two levels up from testbooks)
project_root = os.path.abspath('../..')
sys.path.insert(0, project_root)

# Import the complete red-teaming framework
from redteam import *

print("🎯 Red Team Framework loaded!")
print("📋 Available functions:")
print("   • Configuration: Config, ModelConfig, etc.")
print("   • Models: HuggingFaceRunner, OllamaRunner")
print("   • Execution: run_red_team_batch, quick_test")
print("   • Analysis: visualize_results, analyze_top_candidates")
print("   • Export: export_to_kaggle, create_config_profile")

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Create and customize configuration
cfg = Config()

# MODEL CONFIGURATION - Update these for your setup
cfg.model.model_name = "openai/gpt-oss-20b"  # Update with your model path
cfg.model.backend = "huggingface"  # or "ollama"
cfg.model.device = "cuda"
cfg.model.dtype = "bfloat16"
cfg.model.max_new_tokens = 256  # Start small for testing
cfg.model.temperature = 0.2

# RUN CONFIGURATION
cfg.run.limit_attempts = 20  # Start small for testing
cfg.run.out_dir = "artifacts"

print(f"✅ Configuration created:")
print(f"   Model: {cfg.model.model_name}")
print(f"   Backend: {cfg.model.backend}")
print(f"   Max tokens: {cfg.model.max_new_tokens}")
print(f"   Attempts: {cfg.run.limit_attempts}")

In [ ]:
# =============================================================================
# SYSTEM INITIALIZATION
# =============================================================================

# Initialize the framework
print("🔄 Initializing red-teaming framework...")

# Check framework status
get_framework_status()

# Initialize if needed
if runner is None:
    runner, seed_families = initialize_framework(cfg)

print(f"\n✅ Framework ready!")
print(f"📝 Loaded {len(seed_families)} safety topic families")
print(f"🎯 Ready for red-teaming with {cfg.model.backend} backend")

In [ ]:
# =============================================================================
# GENERATIVE TEST
# =============================================================================
# Run a quick test to ensure everything is working

# Test with proper message format (FIXED)
test_messages = [{"role": "user", "content": "Hello, this is a test prompt."}]

print("🧪 Testing HuggingFaceRunner with generate()...")
try:
    response = runner.generate(test_messages)
    print(f"✅ generate() response: {response}")
    print("✅ HuggingFaceRunner generate() works!")
except Exception as e:
    print(f"❌ generate() test failed: {type(e).__name__}: {e}")
    print("🔧 This indicates a configuration or setup issue.")

print("\n🧪 Testing HuggingFaceRunner with generate_chat()...")
try:
    response = runner.generate_chat(test_messages)
    print(f"✅ generate_chat() response structure: {list(response.keys())}")
    print("\n📋 Detailed response breakdown:")
    for key, value in response.items():
        print(f"  {key}: {value}")
    print("✅ HuggingFaceRunner generate_chat() works!")
except Exception as e:
    print(f"❌ generate_chat() test failed: {type(e).__name__}: {e}")
    print("🔧 This indicates a configuration or setup issue.")

In [ ]:
long_content = "This is a very long message. " * 200  # ~1200 words
test_messages= [{"role": "user", "content": long_content}]

print("\n🧪 Testing long msg HuggingFaceRunner with generate_chat()...")
try:
    response = runner.generate_chat(test_messages)
    print(f"✅ generate_chat() response structure: {list(response.keys())}")
    print("\n📋 Detailed response breakdown:")
    for key, value in response.items():
        print(f"  {key}: {value}")
    print("✅ HuggingFaceRunner generate_chat() works!")
except Exception as e:
    print(f"❌ generate_chat() test failed: {type(e).__name__}: {e}")
    print("🔧 This indicates a configuration or setup issue.")

In [ ]:
print("Model response:", response['gen_text'].split("<|channel|>final<|message|>")[-1].strip())
print("Full Generated Text:", response['gen_text'])

In [ ]:
#!/usr/bin/env python3
"""
Test-Driven Development for Phase 3: Multi-Turn Conversation Framework

This file tests each component as we build it to ensure quality before notebook integration.
"""

import time
import torch
import gc
from typing import List, Dict, Any

class HuggingFaceRunnerTester:
    """Professional test suite for HuggingFaceRunner with comprehensive coverage"""
    
    def __init__(self, runner, cfg):
        self.runner = runner
        self.cfg = cfg
        self.test_results = []
        self.failed_tests = []
        
    def log_test(self, test_name: str, passed: bool, details: str = ""):
        """Log test results"""
        status = "✅ PASS" if passed else "❌ FAIL"
        self.test_results.append({
            'test': test_name,
            'passed': passed,
            'details': details
        })
        print(f"{status} | {test_name}")
        if details and not passed:
            print(f"    Details: {details}")
        if not passed:
            self.failed_tests.append(test_name)

    def test_input_validation(self) -> bool:
        """Test 1: Input validation and error handling"""
        print("\n🔍 Test Group 1: Input Validation")
        all_passed = True
        
        # Test 1.1: String input (should fail)
        try:
            self.runner.generate("This is a string, not a list")
            self.log_test("1.1 String input rejection", False, "Should have raised TypeError")
            all_passed = False
        except TypeError as e:
            if "Expected List[Dict[str, str]]" in str(e):
                self.log_test("1.1 String input rejection", True)
            else:
                self.log_test("1.1 String input rejection", False, f"Wrong error message: {e}")
                all_passed = False
        except Exception as e:
            self.log_test("1.1 String input rejection", False, f"Wrong exception type: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 1.2: Empty list (should fail)
        try:
            self.runner.generate([])
            self.log_test("1.2 Empty list rejection", False, "Should have raised ValueError")
            all_passed = False
        except ValueError as e:
            if "cannot be empty" in str(e):
                self.log_test("1.2 Empty list rejection", True)
            else:
                self.log_test("1.2 Empty list rejection", False, f"Wrong error message: {e}")
                all_passed = False
        except Exception as e:
            self.log_test("1.2 Empty list rejection", False, f"Wrong exception type: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 1.3: Invalid message format (should fail)
        try:
            self.runner.generate([{"invalid": "format"}])
            self.log_test("1.3 Invalid message format rejection", False, "Should have raised ValueError")
            all_passed = False
        except ValueError as e:
            if "missing required 'content' field" in str(e):
                self.log_test("1.3 Invalid message format rejection", True)
            else:
                self.log_test("1.3 Invalid message format rejection", False, f"Wrong error message: {e}")
                all_passed = False
        except Exception as e:
            self.log_test("1.3 Invalid message format rejection", False, f"Wrong exception type: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 1.4: Valid message format (should pass)
        try:
            valid_messages = [{"role": "user", "content": "Test message"}]
            result = self.runner.generate(valid_messages)
            if isinstance(result, str) and len(result) > 0:
                self.log_test("1.4 Valid message format acceptance", True)
            else:
                self.log_test("1.4 Valid message format acceptance", False, f"Invalid response type or empty: {type(result)}")
                all_passed = False
        except Exception as e:
            self.log_test("1.4 Valid message format acceptance", False, f"Unexpected error: {type(e).__name__}: {e}")
            all_passed = False
        
        return all_passed

    def test_message_formats(self) -> bool:
        """Test 2: Different message format scenarios"""
        print("\n🔍 Test Group 2: Message Format Handling")
        all_passed = True
        
        test_cases = [
            {
                "name": "2.1 Single user message",
                "messages": [{"role": "user", "content": "Hello"}]
            },
            {
                "name": "2.2 System + user message",
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": "What is 2+2?"}
                ]
            },
            {
                "name": "2.3 Multi-turn conversation",
                "messages": [
                    {"role": "user", "content": "Tell me about AI"},
                    {"role": "assistant", "content": "AI is artificial intelligence"},
                    {"role": "user", "content": "Tell me more"}
                ]
            },
            {
                "name": "2.4 Message without role field (defaults)",
                "messages": [{"content": "Test without role"}]
            }
        ]
        
        for case in test_cases:
            try:
                result = self.runner.generate(case["messages"])
                if isinstance(result, str) and len(result.strip()) > 0:
                    self.log_test(case["name"], True)
                else:
                    self.log_test(case["name"], False, f"Empty or invalid response: '{result}'")
                    all_passed = False
            except Exception as e:
                self.log_test(case["name"], False, f"Error: {type(e).__name__}: {e}")
                all_passed = False
        
        return all_passed

    def test_token_budget_handling(self) -> bool:
        """Test 3: Token budget and context management"""
        print("\n🔍 Test Group 3: Token Budget Management")
        all_passed = True
        
        # Test 3.1: Short input (should work normally)
        try:
            short_msg = [{"role": "user", "content": "Hi"}]
            result = self.runner.generate_chat(short_msg)
            
            # Validate response structure
            required_fields = ["gen_text", "token_info", "gen_params"]
            missing_fields = [f for f in required_fields if f not in result]
            
            if not missing_fields:
                self.log_test("3.1 Short input processing", True)
            else:
                self.log_test("3.1 Short input processing", False, f"Missing fields: {missing_fields}")
                all_passed = False
        except Exception as e:
            self.log_test("3.1 Short input processing", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 3.2: Very long input (should handle gracefully)
        try:
            long_content = "This is a very long message. " * 200  # ~1200 words
            long_msg = [{"role": "user", "content": long_content}]
            result = self.runner.generate_chat(long_msg)
            
            # Should still work but may have adjusted token budget
            if "gen_text" in result and "token_info" in result:
                token_info = result["token_info"]
                if "dynamic_adjustment" in token_info:
                    self.log_test("3.2 Long input handling", True, 
                                f"Dynamic adjustment: {token_info.get('dynamic_adjustment', 'N/A')}")
                else:
                    self.log_test("3.2 Long input handling", True)
            else:
                self.log_test("3.2 Long input handling", False, "Missing required response fields")
                all_passed = False
        except Exception as e:
            self.log_test("3.2 Long input handling", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        return all_passed

    def test_generation_parameters(self) -> bool:
        """Test 4: Generation parameter handling"""
        print("\n🔍 Test Group 4: Generation Parameters")
        all_passed = True
        
        # Test 4.1: Temperature effects
        try:
            msg = [{"role": "user", "content": "Generate a creative story opening."}]
            
            # Store original temperature
            original_temp = self.cfg.model.temperature
            
            # Test with low temperature
            self.cfg.model.temperature = 0.1
            result_low = self.runner.generate(msg)
            
            # Test with high temperature  
            self.cfg.model.temperature = 0.9
            result_high = self.runner.generate(msg)
            
            # Restore original
            self.cfg.model.temperature = original_temp
            
            # Both should succeed
            if isinstance(result_low, str) and isinstance(result_high, str):
                self.log_test("4.1 Temperature parameter handling", True,
                            f"Low temp: {len(result_low)} chars, High temp: {len(result_high)} chars")
            else:
                self.log_test("4.1 Temperature parameter handling", False, "Non-string results")
                all_passed = False
                
        except Exception as e:
            # Restore original temperature on error
            self.cfg.model.temperature = original_temp
            self.log_test("4.1 Temperature parameter handling", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 4.2: Max tokens parameter
        try:
            msg = [{"role": "user", "content": "Write a detailed explanation of machine learning."}]
            
            original_max_tokens = self.cfg.model.max_new_tokens
            
            # Test with very small token limit
            self.cfg.model.max_new_tokens = 10
            result = self.runner.generate_chat(msg)
            
            # Restore original
            self.cfg.model.max_new_tokens = original_max_tokens
            
            if "gen_text" in result and len(result["gen_text"].strip()) > 0:
                self.log_test("4.2 Max tokens parameter", True, 
                            f"Generated {len(result['gen_text'].split())} words with limit 10 tokens")
            else:
                self.log_test("4.2 Max tokens parameter", False, "Empty or invalid result")
                all_passed = False
                
        except Exception as e:
            self.cfg.model.max_new_tokens = original_max_tokens
            self.log_test("4.2 Max tokens parameter", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        return all_passed

    def test_chat_template_handling(self) -> bool:
        """Test 5: Chat template functionality"""
        print("\n🔍 Test Group 5: Chat Template Handling")
        all_passed = True
        
        # Test 5.1: Harmony chat template enabled
        try:
            original_harmony = self.cfg.model.use_harmony_chat_template
            
            self.cfg.model.use_harmony_chat_template = True
            msg = [{"role": "user", "content": "Test with harmony template"}]
            result_harmony = self.runner.generate_chat(msg)
            
            self.cfg.model.use_harmony_chat_template = False
            result_no_harmony = self.runner.generate_chat(msg)
            
            # Restore original
            self.cfg.model.use_harmony_chat_template = original_harmony
            
            # Both should work
            if ("gen_text" in result_harmony and "gen_text" in result_no_harmony and
                len(result_harmony["gen_text"].strip()) > 0 and len(result_no_harmony["gen_text"].strip()) > 0):
                self.log_test("5.1 Chat template toggle", True,
                            f"Harmony: {len(result_harmony['gen_text'])} chars, No harmony: {len(result_no_harmony['gen_text'])} chars")
            else:
                self.log_test("5.1 Chat template toggle", False, "Invalid results from template comparison")
                all_passed = False
                
        except Exception as e:
            self.cfg.model.use_harmony_chat_template = original_harmony
            self.log_test("5.1 Chat template toggle", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        return all_passed

    def test_error_handling_robustness(self) -> bool:
        """Test 6: Error handling and edge cases"""
        print("\n🔍 Test Group 6: Error Handling & Edge Cases")
        all_passed = True
        
        # Test 6.1: Unicode and special characters
        try:
            unicode_msg = [{"role": "user", "content": "Hello! 👋 Can you handle émojis and spëciál châractërs? 中文 العربية"}]
            result = self.runner.generate(unicode_msg)
            if isinstance(result, str) and len(result.strip()) > 0:
                self.log_test("6.1 Unicode character handling", True)
            else:
                self.log_test("6.1 Unicode character handling", False, "Invalid unicode response")
                all_passed = False
        except Exception as e:
            self.log_test("6.1 Unicode character handling", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 6.2: Very short content
        try:
            short_msg = [{"role": "user", "content": "?"}]
            result = self.runner.generate(short_msg)
            if isinstance(result, str):
                self.log_test("6.2 Minimal input handling", True, f"Response: '{result[:50]}...'")
            else:
                self.log_test("6.2 Minimal input handling", False, "Invalid response type")
                all_passed = False
        except Exception as e:
            self.log_test("6.2 Minimal input handling", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 6.3: Memory cleanup (basic check)
        try:
            initial_memory = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
            
            # Generate several responses
            for i in range(3):
                msg = [{"role": "user", "content": f"Memory test {i}"}]
                _ = self.runner.generate(msg)
            
            # Force cleanup
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            final_memory = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
            memory_diff = final_memory - initial_memory
            
            # Memory should not grow excessively
            self.log_test("6.3 Memory usage stability", True, 
                         f"Memory change: {memory_diff:,} bytes ({memory_diff/1024/1024:.2f} MB)")
                         
        except Exception as e:
            self.log_test("6.3 Memory usage stability", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        return all_passed

    def test_performance_benchmarks(self) -> bool:
        """Test 7: Basic performance benchmarks"""
        print("\n🔍 Test Group 7: Performance Benchmarks")
        all_passed = True
        
        # Test 7.1: Generation speed
        try:
            msg = [{"role": "user", "content": "Write a short paragraph about technology."}]
            
            start_time = time.time()
            result = self.runner.generate(msg)
            end_time = time.time()
            
            generation_time = end_time - start_time
            tokens_generated = len(result.split()) if isinstance(result, str) else 0
            tokens_per_second = tokens_generated / generation_time if generation_time > 0 else 0
            
            if isinstance(result, str) and len(result.strip()) > 0:
                self.log_test("7.1 Generation speed", True, 
                             f"Time: {generation_time:.2f}s, Tokens: {tokens_generated}, Speed: {tokens_per_second:.1f} tok/s")
            else:
                self.log_test("7.1 Generation speed", False, "Invalid generation result")
                all_passed = False
                
        except Exception as e:
            self.log_test("7.1 Generation speed", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        return all_passed

    def test_multi_turn_conversations(self) -> bool:
        """Test 8: Multi-turn conversation functionality"""
        print("\n🔍 Test Group 8: Multi-turn Conversations")
        all_passed = True
        
        # Import ConversationManager for testing
        try:
            from redteam.test_conversation_framework import ConversationManager
        except ImportError:
            # Create minimal mock if import fails
            class ConversationManager:
                def __init__(self, max_turns=3):
                    self.max_turns = max_turns
                    self.conversations = {}
                
                def create_conversation(self, seed_prompt):
                    import uuid
                    conv_id = f"conv_{uuid.uuid4().hex[:8]}"
                    self.conversations[conv_id] = {
                        'id': conv_id, 'turns': [], 'seed': seed_prompt
                    }
                    return conv_id
                
                def add_turn(self, conv_id, prompt, response):
                    if conv_id in self.conversations:
                        self.conversations[conv_id]['turns'].append({
                            'prompt': prompt, 'response': response
                        })
        
        # Test 8.1: Basic 2-turn conversation flow
        try:
            conversation_manager = ConversationManager(max_turns=3)
            
            # Create conversation
            conv_id = conversation_manager.create_conversation("Test seed")
            
            # Turn 1: Initial prompt
            turn1_msg = [{"role": "user", "content": "Tell me about AI safety"}]
            result1 = self.runner.generate_chat(turn1_msg)
            conversation_manager.add_turn(conv_id, turn1_msg[0]["content"], result1["gen_text"])
            
            # Turn 2: Follow-up based on response
            turn2_msg = [
                {"role": "user", "content": "Tell me about AI safety"},
                {"role": "assistant", "content": result1["gen_text"]},
                {"role": "user", "content": "Can you elaborate on potential risks?"}
            ]
            result2 = self.runner.generate_chat(turn2_msg)
            conversation_manager.add_turn(conv_id, turn2_msg[-1]["content"], result2["gen_text"])
            
            if len(conversation_manager.conversations[conv_id]["turns"]) == 2:
                self.log_test("8.1 Basic 2-turn conversation", True, 
                            f"Successfully created conversation with 2 turns")
            else:
                self.log_test("8.1 Basic 2-turn conversation", False, 
                            f"Expected 2 turns, got {len(conversation_manager.conversations[conv_id]['turns'])}")
                all_passed = False
                
        except Exception as e:
            self.log_test("8.1 Basic 2-turn conversation", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 8.2: Extended 3-turn conversation  
        try:
            conv_id2 = conversation_manager.create_conversation("Extended test")
            responses = []
            
            # Simulate 3 turns
            for turn_num in range(1, 4):
                if turn_num == 1:
                    messages = [{"role": "user", "content": f"Turn {turn_num}: What is machine learning?"}]
                else:
                    # Build conversation history
                    messages = []
                    for prev_turn in conversation_manager.conversations[conv_id2]["turns"]:
                        messages.append({"role": "user", "content": prev_turn["prompt"]})
                        messages.append({"role": "assistant", "content": prev_turn["response"]})
                    messages.append({"role": "user", "content": f"Turn {turn_num}: Tell me more details"})
                
                result = self.runner.generate_chat(messages)
                responses.append(result)
                conversation_manager.add_turn(conv_id2, messages[-1]["content"], result["gen_text"])
            
            if len(conversation_manager.conversations[conv_id2]["turns"]) == 3:
                self.log_test("8.2 Extended 3-turn conversation", True,
                            f"Successfully created 3-turn conversation")
            else:
                self.log_test("8.2 Extended 3-turn conversation", False, 
                            f"Expected 3 turns, got {len(conversation_manager.conversations[conv_id2]['turns'])}")
                all_passed = False
                
        except Exception as e:
            self.log_test("8.2 Extended 3-turn conversation", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 8.3: Conversation context preservation
        try:
            # Create conversation with specific context that should be preserved
            conv_id3 = conversation_manager.create_conversation("Context test")
            
            # Turn 1: Establish context
            context_msg = [{"role": "user", "content": "My name is Alice and I work in cybersecurity"}]
            result1 = self.runner.generate_chat(context_msg)
            conversation_manager.add_turn(conv_id3, context_msg[0]["content"], result1["gen_text"])
            
            # Turn 2: Reference previous context
            followup_msg = [
                {"role": "user", "content": "My name is Alice and I work in cybersecurity"},
                {"role": "assistant", "content": result1["gen_text"]},
                {"role": "user", "content": "What security advice would you give someone in my field?"}
            ]
            result2 = self.runner.generate_chat(followup_msg)
            
            # Check if context appears to be maintained (basic heuristic)
            context_preserved = (
                len(result2["gen_text"]) > 20 and  # Response has substance
                "gen_text" in result2 and  # Valid response structure
                len(followup_msg) == 3  # Conversation history properly built
            )
            
            if context_preserved:
                self.log_test("8.3 Conversation context preservation", True,
                            f"Context appears preserved in {len(result2['gen_text'])} char response")
            else:
                self.log_test("8.3 Conversation context preservation", False, 
                            "Context preservation validation failed")
                all_passed = False
                
        except Exception as e:
            self.log_test("8.3 Conversation context preservation", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        # Test 8.4: ConversationManager integration
        try:
            # Test conversation manager functionality
            total_conversations = len(conversation_manager.conversations)
            
            # Verify all test conversations were created
            expected_conversations = 3  # conv_id, conv_id2, conv_id3
            
            if total_conversations >= expected_conversations:
                self.log_test("8.4 ConversationManager integration", True,
                            f"Created {total_conversations} conversations successfully")
            else:
                self.log_test("8.4 ConversationManager integration", False,
                            f"Expected ≥{expected_conversations} conversations, got {total_conversations}")
                all_passed = False
                
        except Exception as e:
            self.log_test("8.4 ConversationManager integration", False, f"Error: {type(e).__name__}: {e}")
            all_passed = False
        
        return all_passed

    def run_all_tests(self) -> Dict[str, Any]:
        """Run the complete test suite"""
        print("🧪 STARTING COMPREHENSIVE HUGGINGFACE RUNNER TEST SUITE")
        print("=" * 70)
        print(f"🎯 Target Model: {self.cfg.model.model_name}")
        print(f"🔧 Backend: {self.cfg.model.backend}")
        print(f"💾 Device: {self.cfg.model.device}")
        print(f"📊 Max Tokens: {self.cfg.model.max_new_tokens}")
        print("=" * 70)
        
        start_time = time.time()
        
        # Run all test groups
        test_groups = [
            ("Input Validation", self.test_input_validation),
            ("Message Formats", self.test_message_formats),
            ("Token Budget", self.test_token_budget_handling),
            ("Generation Parameters", self.test_generation_parameters),
            ("Chat Templates", self.test_chat_template_handling),
            ("Error Handling", self.test_error_handling_robustness),
            ("Performance", self.test_performance_benchmarks),
            ("Multi-turn Conversations", self.test_multi_turn_conversations)
        ]
        
        group_results = {}
        for group_name, test_func in test_groups:
            try:
                group_results[group_name] = test_func()
            except Exception as e:
                print(f"\n❌ CRITICAL ERROR in {group_name}: {type(e).__name__}: {e}")
                group_results[group_name] = False
        
        end_time = time.time()
        total_time = end_time - start_time
        
        # Generate summary
        print(f"\n🏁 TEST SUITE COMPLETE")
        print("=" * 70)
        print(f"⏱️  Total runtime: {total_time:.2f} seconds")
        print(f"📊 Tests run: {len(self.test_results)}")
        
        passed_count = sum(1 for result in self.test_results if result['passed'])
        failed_count = len(self.test_results) - passed_count
        
        print(f"✅ Passed: {passed_count}")
        print(f"❌ Failed: {failed_count}")
        
        if failed_count == 0:
            print("🎉 ALL TESTS PASSED - HuggingFaceRunner is working correctly!")
        else:
            print(f"⚠️  {failed_count} test(s) failed. See details above.")
            print(f"🔧 Failed tests: {', '.join(self.failed_tests)}")
        
        success_rate = (passed_count / len(self.test_results)) * 100 if self.test_results else 0
        print(f"📈 Success rate: {success_rate:.1f}%")
        
        return {
            "total_tests": len(self.test_results),
            "passed": passed_count,
            "failed": failed_count,
            "success_rate": success_rate,
            "runtime": total_time,
            "failed_tests": self.failed_tests,
            "group_results": group_results,
            "detailed_results": self.test_results
        }

# Execute the comprehensive test suite
print("🚀 Initializing HuggingFaceRunner Test Suite...")

if 'runner' in globals() and runner is not None:
    tester = HuggingFaceRunnerTester(runner, cfg)
    test_summary = tester.run_all_tests()
    
    # Store test results for future reference
    globals()['hf_runner_test_results'] = test_summary
    
    if test_summary['success_rate'] >= 90:
        print("\n🏆 EXCELLENT: HuggingFaceRunner passes professional testing standards!")
    elif test_summary['success_rate'] >= 75:
        print("\n👍 GOOD: HuggingFaceRunner is functional with minor issues.")
    else:
        print("\n⚠️  ATTENTION NEEDED: Multiple test failures require investigation.")
        
else:
    print("❌ Runner not initialized. Please run the framework initialization cell first.")